# Decision Tree Analysis

#### DOUBTS:
- Cell with value: 999999999
- Have to normalize? Think not

Importing necessary packages:

In [1]:
import os
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

UTILS:

In [52]:
DATABASE_DIR = '/home/macdowell/Workspace/Decision-Tree-Analysis/database/'

def get_entries_name_inside_dir(path):
    smell_folders = os.listdir(path)
    return smell_folders

def dataframe_appended_from_smell_folder(folder_path):
    csv_files = os.listdir(folder_path)
    df = None
    
    for csv in csv_files:
        if 'lock' in csv:
            continue
        
        if df is None:
            df = pd.read_csv(folder_path + '/' + csv)
        else:
            df_aux = pd.read_csv(folder_path + '/' + csv)
            df = pd.concat([df, df_aux])
    return df

### Effectivity on different database's percents:

- RQ1)

In [57]:
smell_folders = get_entries_name_inside_dir(DATABASE_DIR)
for smell in smell_folders:
    smell_metrics_df = dataframe_appended_from_smell_folder(DATABASE_DIR + smell)
    
    smell_metrics_df[smell_metrics_df['Smell'] == False] = 0
    smell_metrics_df[smell_metrics_df['Smell'] == True] = 1
    print(smell_metrics_df['Smell'])
    
    X = smell_metrics_df.iloc[:, smell_metrics_df.columns != 'Smell']
    y = smell_metrics_df.iloc[:, smell_metrics_df.columns == 'Smell']
    
    
    break

0     0
1     0
2     0
3     0
4     0
5     0
6     1
7     1
8     0
9     0
10    0
11    0
12    0
13    1
14    0
0     0
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     0
9     0
10    0
11    1
12    1
13    1
14    0
     ..
0     1
1     1
2     0
3     0
4     0
5     1
6     0
7     1
8     1
9     0
10    0
11    0
12    1
13    1
14    1
0     0
1     0
2     0
3     0
4     0
5     0
6     1
7     1
8     0
9     0
10    0
11    1
12    1
13    1
14    1
Name: Smell, Length: 180, dtype: int64


/home/macdowell/Programs/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



### The maximum depth of the tree:

- RQ2)

### Effectivity on different database's percents with different maximum depth of the tree: